In [62]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img
import tensorflow as tf
import numpy as np
import cv2
import mediapipe as mp
import paho.mqtt.client as mqtt
move = 0
client = mqtt.Client()
# Cargar el modelo desde el archivo H5
model = load_model('modelo1_5classes.h5')

In [63]:
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+str(rc))
    client.subscribe("proyecto/estado")

def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload))

In [64]:
client.on_connect = on_connect
client.on_message = on_message


In [65]:


#-------------------------CODIGO PARA GRABACION DEL VIDEO----------------------------------

def grabar_video():
    # Establecer las dimensiones deseadas del video
    width = 640
    height = 396

    # Inicializar la captura de video desde la webcam
    video_capture = cv2.VideoCapture(0)
    
    # Establecer el tamaño del cuadro de captura
    video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

    # Obtener la información de tamaño del video
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30  # FPS (cuadros por segundo) del video

    # Crear el objeto de salida de video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_out = cv2.VideoWriter('video_salida.mp4', fourcc, fps, (width, height))

    # Grabar video durante 5 segundos
    tiempo_inicio = cv2.getTickCount()  # Obtener el tiempo actual en ticks
    duracion_grabacion = 3  # Duración de la grabación en segundos

    while True:
        # Capturar un cuadro de video
        ret, frame = video_capture.read()

        if ret:
            # Escribir el cuadro en el archivo de salida
            video_out.write(frame)

            # Calcular el tiempo transcurrido
            tiempo_actual = cv2.getTickCount()
            tiempo_transcurrido = (tiempo_actual - tiempo_inicio) / cv2.getTickFrequency()

            # Detener la grabación después de 5 segundos
            if tiempo_transcurrido >= duracion_grabacion:
                break

        # Mostrar el cuadro de video en una ventana
        cv2.imshow('Video', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Liberar los recursos
    video_capture.release()
    video_out.release()
    cv2.destroyAllWindows()

# Llamar a la función para grabar el video
grabar_video()

#------------------------------------------------------------------------------------
#-----------------------------CODIGO PARA SEGMENTACION DE PIEL ----------------------
def equaliseCLAHEOpenCV(image):
    image_base = np.array(image, copy=True)

  # Convert if it's an RGB image
    if len(image_base.shape) > 2:
        image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)[:,:,0]

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(9,9))
    cl1 = clahe.apply(image)

  # Inverse conversion if it was an RGB image
    if len(image_base.shape) > 2:
        image_aux = cv2.cvtColor(image_base, cv2.COLOR_RGB2YUV)
        image_aux[:,:,0] = cl1
        result = cv2.cvtColor(image_aux,cv2.COLOR_YUV2RGB)
  
    return result


def segmentar_piel(frame):
    # Aplicar mejoramiento de contraste
    frame = equaliseCLAHEOpenCV(frame)
    ycbcr = cv2.cvtColor(frame, cv2.COLOR_BGR2YCrCb)

    # Definir los rangos de color para la piel en el espacio de color YCbCr
    bajo = np.array([0, 133, 50], dtype=np.uint8)
    alto = np.array([255, 173, 200], dtype=np.uint8)

    # Crear la máscara de piel en el espacio de color YCbCr
    mascara_piel = cv2.inRange(ycbcr, bajo, alto)
    #IRMPRIIR ALGO SI MARCARA PIEL ESTA VACIO O NO
    if mascara_piel.sum() == 0:
        print("No se detecto piel")

    # Aplicar la máscara a la imagen original
    frame_sinfondo = frame.copy()
    frame_sinfondo[mascara_piel == 0] = [0, 0, 0]

    return frame_sinfondo


def preprocess_video(input_video_path, output_video_path, target_resolution=(704, 396), target_frames=100):
    video = cv2.VideoCapture(input_video_path)
    frames_total = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    sampling_interval = max(frames_total // target_frames, 1)
    selected_frames = []

    for frame_idx in range(frames_total):
        ret, frame = video.read()

        if frame_idx % sampling_interval == 0:
            frame = cv2.resize(frame, target_resolution)
            frame = segmentar_piel(frame)
            selected_frames.append(frame)

    video.release()
    selected_frames = np.array(selected_frames)

    if selected_frames.shape[0] < target_frames:
        selected_frames = np.pad(selected_frames, [(0, target_frames - selected_frames.shape[0]), (0, 0), (0, 0), (0, 0)], mode='wrap')
    else:
        selected_frames = selected_frames[:target_frames]

    dataset = tf.data.Dataset.from_tensor_slices(selected_frames)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = cv2.VideoWriter(output_video_path, fourcc, fps, target_resolution)

    for frame in dataset:
        frame = frame.numpy().astype(np.uint8)
        output_video.write(frame)

    output_video.release()




# Ejemplo de uso
#Se pone como primer parametro el video de entrada
#Se pone como segundo parametro el nombre del video de salida
preprocess_video('video_salida.mp4', 'video_piel.mp4')

#------------------------------------------------------------------------------------



#-----------------------------CODIGO PARA DETECCION DE MANOS ------------------------
# Cargar el módulo de Mediapipe para detección de manos
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# Cargar el video
video_path = "video_piel.mp4"
cap = cv2.VideoCapture(video_path)

# Configurar la salida de la imagen resumen
summary_output_path = "salida.jpg"

# Configurar los puntos clave que se desean mostrar
desired_keypoints = [5, 9, 13]  # Índices de los puntos clave de interés


# Configurar Mediapipe para la detección de manos
with mp_hands.Hands(static_image_mode=True, max_num_hands=2,
                    min_detection_confidence=0.05, min_tracking_confidence=0.03) as hands:
    frame_count = 0
    summary_image = np.zeros((396, 640, 3))

    while cap.isOpened():
        # Leer el siguiente frame
        success, image = cap.read()
        #redimensionar la imagen
        image = cv2.resize(image, (640, 396))
        #image=cv2.re
        black_image = np.zeros_like(image)
        colors = [(0, 0, 255), (0, 255, 0), (255, 0, 0)] 
        if not success:
            break



        # Detección de manos
        results_hands = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))


        if results_hands.multi_hand_landmarks:

            for hand_landmarks in results_hands.multi_hand_landmarks:
                for idx, landmark in enumerate(hand_landmarks.landmark):
                    if idx in desired_keypoints:
                        x = int(landmark.x * image.shape[1])
                        y = int(landmark.y * image.shape[0])

                        color = colors[desired_keypoints.index(idx)]  # Obtener el color correspondiente al punto clave
                        cv2.circle(black_image, (x, y), 2, color, -1)


            summary_image += black_image.astype(np.float32)

        frame_count += 1   
        if frame_count == 100:
            break
    
    summary_image = np.clip(summary_image, 0, 255).astype(np.uint8)
    cv2.imwrite(summary_output_path, summary_image)
# Liberar recursos
cap.release()

#------------------------------------------------------------------------------------
#-----------------------------CODIGO PARA CLASIFICACION DE GESTO -----------------------


# Preprocesar la imagen de entrada
img = load_img('salida.jpg')
img = tf.image.resize(img, (640, 396))
test = np.array([img])

# Realizar la predicción utilizando el modelo cargado
predictions = model.predict(test)

# Interpretar las predicciones
#labels = ['CUANDO', 'DERECHA', 'DISMINUIR', 'IZQUIERDA', 'SI']
labels = ['AVANZAR', 'DERECHA', 'DISMINUIR', 'IZQUIERDA', 'AUMENTAR']
predicted_labels = [labels[np.argmax(prediction)] for prediction in predictions]

print(predicted_labels)




1/1 [==============================] - 0s 125ms/step
['AUMENTAR']


In [67]:
client.connect("mqtt-dashboard.com", 1883, 60)
client.loop_start()
if predicted_labels[0] == 'AVANZAR':
    print("x")
    if move == 0:
        print("y")
        client.publish("proyecto/estado", "l/1")
        move=1
    elif move == 1:
        print("z")
        client.publish("proyecto/estado", "l/4")
        move=0
elif predicted_labels[0] == 'DERECHA':
    client.publish("proyecto/estado", "l/2")
elif predicted_labels[0] == 'IZQUIERDA':
    client.publish("proyecto/estado", "l/3")
elif predicted_labels[0] == 'AUMENTAR':
    client.publish("proyecto/estado", "l/5")
elif predicted_labels[0] == 'DISMINUIR':
    client.publish("proyecto/estado", "l/6")
client.loop_stop()